In [ ]:
import numpy as np
from numpy.ma import masked_values as mv
import gsw
import xarray as xr
import pandas as pd
import os.path as op
from datetime import datetime, timedelta
from scipy.interpolate import PchipInterpolator as pchip
from scipy.interpolate import Akima1DInterpolator as akima
from scipy.signal import medfilt
import dask.array as dsar
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib
from matplotlib import cm
import matplotlib as mpl
import matplotlib.colors as clr
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker # buat ganti tick
import scipy
import math 
from scipy.signal import find_peaks
np.warnings.filterwarnings('ignore') # ini buat ignore si warning nan
import datetime

%matplotlib inline

# open the required dataset
### change the nc file within the apostrophe (red color) into your smoothed ndvi time series dataset

In [ ]:
ds = xr.open_dataset('ndvi_agriculture_2009based.nc').ndvi

In [ ]:
ds.load()

# define the range of the month for start, peak, and end
### the range of month and the total number of cropping season will depend on the general cropping season in each region
### in this case, because there are two dry cropping season, there are also two of the start, peak, and end season

In [ ]:
def start_month_1(month):
    return (month >= 1) & (month <= (int(max_month_1)))

In [ ]:
def peak_month_1(month):
    return (month >= 1) & (month <= 8)

In [ ]:
def end_month_1(month):
    return (month >= (int(max_month_1))) & (month <= 12)

In [ ]:
def start_month_2(month):
    return (month >= 1) & (month <= (int(max_month_2)))

In [ ]:
def peak_month_2(month):
    return (month >= 1) & (month <= 8)

In [ ]:
def end_month_2(month):
    return (month >= (int(max_month_2))) & (month <= 12)

### create a storage for saving data during looping process
#### notes: the number "21" comes from total year in study period; because this study conducted from 2001 - 2021 so the total times is 21. please adjust it accordingly to your study period.

In [ ]:
storage_season = np.zeros([len(ds.lat),len(ds.lon),21])

storage_pos_1 = np.zeros([len(ds.lat),len(ds.lon),21])
storage_sos_1 = np.zeros([len(ds.lat),len(ds.lon),21])
storage_eos_1 = np.zeros([len(ds.lat),len(ds.lon),21])

storage_pos_2 = np.zeros([len(ds.lat),len(ds.lon),21])
storage_sos_2 = np.zeros([len(ds.lat),len(ds.lon),21])
storage_eos_2 = np.zeros([len(ds.lat),len(ds.lon),21])

# main code and looping

In [ ]:
%%time

for i in range(len(ds.lat)):
    if i%10 == 0:
        print(f'currently working on the latitude - {i}')
    for j in range(len(ds.lon)):
        
        current_grid = ds.isel(lat=i, lon=j)
        
        if np.isnan(current_grid).any()==False:

            for k in range(0,21):

                current_year = current_grid.sel(time = f"{2001+k}")

                peak_year_1 = current_year.sel(time = peak_month_1(current_year['time.month']))
                peak_index_1 = (np.diff(np.sign(np.diff(peak_year_1))) < 0).nonzero()[0] + 1
                
                storage_season[i,j,k] = (len(peak_index_1))

                if (len(peak_index_1) == 1):

                    peak_date_1 = peak_year_1[peak_index_1].time
                    max_month_1 = peak_date_1.time.dt.strftime("%m")

                    start_year_1 = current_year.sel(time = start_month_1(current_year['time.month']))
                    start_index_1 = (np.diff(np.sign(np.diff(start_year_1))) > 0).nonzero()[0] + 1

                    if (len(start_index_1) > 0):

                        end_year_1 = current_year.sel(time = end_month_1(current_year['time.month']))
                        end_index_1 = (np.diff(np.sign(np.diff(end_year_1))) > 0).nonzero()[0] + 1

                        if (len(end_index_1) > 0):

                            storage_pos_1[i,j,k] = peak_date_1.dt.dayofyear
                            storage_pos_2[i,j,k] = np.nan

                            start_date_1 = start_year_1[(start_index_1.max())].time
                            storage_sos_1[i,j,k] = start_date_1.dt.dayofyear
                            storage_sos_2[i,j,k] = np.nan

                            end_date_1 = end_year_1[(end_index_1.min())].time
                            storage_eos_1[i,j,k] = end_date_1.dt.dayofyear
                            storage_eos_2[i,j,k] = np.nan

                if (len(peak_index_1) > 1):

                    peak_date_1 = peak_year_1[(peak_index_1.min())].time
                    peak_date_2 = peak_year_1[(peak_index_1[1])].time
                    max_month_1 = peak_date_1.time.dt.strftime("%m")
                    max_month_2 = peak_date_2.time.dt.strftime("%m")

                    start_year_1 = current_year.sel(time = start_month_1(current_year['time.month']))
                    start_index_1 = (np.diff(np.sign(np.diff(start_year_1))) > 0).nonzero()[0] + 1
                    start_year_2 = current_year.sel(time = start_month_2(current_year['time.month']))
                    start_index_2 = (np.diff(np.sign(np.diff(start_year_2))) > 0).nonzero()[0] + 1

                    if (len(start_index_1) == 0) & (len(start_index_2) > 0):

                        end_year_2 = current_year.sel(time = end_month_2(current_year['time.month']))
                        end_index_2 = (np.diff(np.sign(np.diff(end_year_2))) > 0).nonzero()[0] + 1
                        
                        if (len(end_index_2) > 0):

                            storage_pos_1[i,j,k] = np.nan
                            storage_pos_2[i,j,k] = peak_date_2.dt.dayofyear

                            storage_sos_1[i,j,k] = np.nan
                            start_date_2 = start_year_2[(start_index_2.max())].time
                            storage_sos_2[i,j,k] = start_date_2.dt.dayofyear

                            storage_eos_1[i,j,k] = np.nan
                            end_date_2 = end_year_2[(end_index_2.min())].time
                            storage_eos_2[i,j,k] = end_date_2.dt.dayofyear

                        
                        if (len(end_index_2) == 0):

                            storage_pos_1[i,j,k] = np.nan
                            storage_pos_2[i,j,k] = np.nan

                            storage_sos_1[i,j,k] = np.nan
                            storage_sos_2[i,j,k] = np.nan

                            storage_eos_1[i,j,k] = np.nan
                            storage_eos_2[i,j,k] = np.nan


                    if (len(start_index_1) > 0) & (len(start_index_2) == 0):

                        end_year_1 = current_year.sel(time = end_month_1(current_year['time.month']))
                        end_index_1 = (np.diff(np.sign(np.diff(end_year_1))) > 0).nonzero()[0] + 1

                        if (len(end_index_1) > 0):
                            
                            storage_pos_1[i,j,k] = peak_date_1.dt.dayofyear
                            storage_pos_2[i,j,k] = np.nan

                            start_date_1 = start_year_1[(start_index_1.max())].time
                            storage_sos_1[i,j,k] = start_date_1.dt.dayofyear
                            storage_sos_2[i,j,k] = np.nan

                            end_date_1 = end_year_1[(end_index_1.min())].time
                            storage_eos_1[i,j,k] = end_date_1.dt.dayofyear
                            storage_eos_2[i,j,k] = np.nan

                        if (len(end_index_1) == 0):
                            
                            storage_pos_1[i,j,k] = np.nan
                            storage_pos_2[i,j,k] = np.nan

                            storage_sos_1[i,j,k] = np.nan
                            storage_sos_2[i,j,k] = np.nan

                            storage_eos_1[i,j,k] = np.nan
                            storage_eos_2[i,j,k] = np.nan


                    if (len(start_index_1) > 0) & (len(start_index_2) > 0):

                        end_year_1 = current_year.sel(time = end_month_1(current_year['time.month']))
                        end_index_1 = (np.diff(np.sign(np.diff(end_year_1))) > 0).nonzero()[0] + 1
                        end_year_2 = current_year.sel(time = end_month_2(current_year['time.month']))
                        end_index_2 = (np.diff(np.sign(np.diff(end_year_2))) > 0).nonzero()[0] + 1

                        if (len(end_index_1) == 0) & (len(end_index_2) > 0):

                            storage_pos_1[i,j,k] = np.nan
                            storage_pos_2[i,j,k] = peak_date_2.dt.dayofyear

                            storage_sos_1[i,j,k] = np.nan
                            start_date_2 = start_year_2[(start_index_2.max())].time
                            storage_sos_2[i,j,k] = start_date_2.dt.dayofyear

                            storage_eos_1[i,j,k] = np.nan
                            end_date_2 = end_year_2[(end_index_2.min())].time
                            storage_eos_2[i,j,k] = end_date_2.dt.dayofyear


                    if (len(start_index_1) > 0) & (len(start_index_2) > 0):

                        end_year_1 = current_year.sel(time = end_month_1(current_year['time.month']))
                        end_index_1 = (np.diff(np.sign(np.diff(end_year_1))) > 0).nonzero()[0] + 1
                        end_year_2 = current_year.sel(time = end_month_2(current_year['time.month']))
                        end_index_2 = (np.diff(np.sign(np.diff(end_year_2))) > 0).nonzero()[0] + 1

                        if (len(end_index_1) > 0) & (len(end_index_2) == 0):

                            storage_pos_1[i,j,k] = peak_date_1.dt.dayofyear
                            storage_pos_2[i,j,k] = np.nan

                            start_date_1 = start_year_1[(start_index_1.max())].time
                            storage_sos_1[i,j,k] = start_date_1.dt.dayofyear
                            storage_sos_2[i,j,k] = np.nan

                            end_date_1 = end_year_1[(end_index_1.min())].time
                            storage_eos_1[i,j,k] = end_date_1.dt.dayofyear
                            storage_eos_2[i,j,k] = np.nan

                            
                    if (len(start_index_1) > 0) & (len(start_index_2) > 0):

                        end_year_1 = current_year.sel(time = end_month_1(current_year['time.month']))
                        end_index_1 = (np.diff(np.sign(np.diff(end_year_1))) > 0).nonzero()[0] + 1
                        end_year_2 = current_year.sel(time = end_month_2(current_year['time.month']))
                        end_index_2 = (np.diff(np.sign(np.diff(end_year_2))) > 0).nonzero()[0] + 1

                        if (len(end_index_1) == 0) & (len(end_index_2) == 0):

                            storage_pos_1[i,j,k] = np.nan
                            storage_pos_2[i,j,k] = np.nan

                            storage_sos_1[i,j,k] = np.nan
                            storage_sos_2[i,j,k] = np.nan

                            storage_eos_1[i,j,k] = np.nan
                            storage_eos_2[i,j,k] = np.nan


                    if (len(start_index_1) > 0) & (len(start_index_2) > 0):

                        end_year_1 = current_year.sel(time = end_month_1(current_year['time.month']))
                        end_index_1 = (np.diff(np.sign(np.diff(end_year_1))) > 0).nonzero()[0] + 1
                        end_year_2 = current_year.sel(time = end_month_2(current_year['time.month']))
                        end_index_2 = (np.diff(np.sign(np.diff(end_year_2))) > 0).nonzero()[0] + 1

                        if (len(end_index_1) > 0) & (len(end_index_2) > 0):

                            storage_pos_1[i,j,k] = peak_date_1.dt.dayofyear
                            storage_pos_2[i,j,k] = peak_date_2.dt.dayofyear

                            start_date_1 = start_year_1[(start_index_1.max())].time
                            storage_sos_1[i,j,k] = start_date_1.dt.dayofyear
                            start_date_2 = start_year_2[(start_index_2.max())].time
                            storage_sos_2[i,j,k] = start_date_2.dt.dayofyear

                            end_date_1 = end_year_1[(end_index_1.min())].time
                            storage_eos_1[i,j,k] = end_date_1.dt.dayofyear
                            end_date_2 = end_year_2[(end_index_2.min())].time
                            storage_eos_2[i,j,k] = end_date_2.dt.dayofyear


# create a dataset

In [ ]:
list_year = np.arange(2001,2022)

In [ ]:
for_year = np.atleast_1d(list_year)

In [ ]:
ds_result = xr.Dataset({'sos_1':(('lat','lon','time'),storage_sos_1),
                        'pos_1':(('lat','lon','time'),storage_pos_1),
                        'eos_1':(('lat','lon','time'),storage_eos_1),
                        'sos_2':(('lat','lon','time'),storage_sos_2),
                        'pos_2':(('lat','lon','time'),storage_pos_2),
                        'eos_2':(('lat','lon','time'),storage_eos_2),
                        'total_season':(('lat','lon','time'),storage_season)},
                        coords = {'lat':ds.lat, 'lon':ds.lon, 'time':for_year})

ds_result = ds_result.where(ds_result>0)

# saving file into .nc format
### change the name within the apostrophe (red color) into your desired dataset name

In [ ]:
ds_result.to_netcdf("annual_crop_calendar.nc")